In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import cv2


In [2]:
train = pd.read_csv('./data/written_name_train_v2.csv')
valid = pd.read_csv('./data/written_name_validation_v2.csv')

In [3]:
train.head

<bound method NDFrame.head of                 FILENAME       IDENTITY
0        TRAIN_00001.jpg      BALTHAZAR
1        TRAIN_00002.jpg          SIMON
2        TRAIN_00003.jpg          BENES
3        TRAIN_00004.jpg        LA LOVE
4        TRAIN_00005.jpg         DAPHNE
...                  ...            ...
330956  TRAIN_330957.jpg          LENNY
330957  TRAIN_330958.jpg        TIFFANY
330958  TRAIN_330959.jpg  COUTINHO DESA
330959  TRAIN_330960.jpg         MOURAD
330960  TRAIN_330961.jpg        HELOISE

[330961 rows x 2 columns]>

In [4]:
#Cleaning the data 
print("Number of train NaN:",train['IDENTITY'].isnull().sum())
print("Number of train NaN:",valid['IDENTITY'].isnull().sum())

Number of train NaN: 565
Number of train NaN: 78


In [5]:
#Dropping NaN Values 
train.dropna(axis=0,inplace=True)
valid.dropna(axis=0,inplace=True)

In [6]:
#Checking for unreadable images 
unreadable = train[train['IDENTITY'] == 'UNREADABLE']
unreadable.reset_index(inplace = True, drop=True)
train = train[train['IDENTITY'] != 'UNREADABLE']
valid = valid[valid['IDENTITY'] != 'UNREADABLE']

In [7]:
#Converting the lowercase to uppercase to maintain uniformity
train['IDENTITY']=train['IDENTITY'].str.upper()
valid['IDENTITY']=valid['IDENTITY'].str.upper()

In [8]:
#Reseting the index of elements
train.reset_index(inplace=True,drop=True)
valid.reset_index(inplace=True,drop=True)

In [9]:
#PREPROCESSING AND PREPARING THE IMAGES

#Image loaded as grayscale and reshaped to 256 and 64
#The width and height are cropped if they are greater than 256 and 64 respectively. If they are smaller, then the image is padded with white pixels. Finally the image is rotated clockwise to bring the image shape to (x, y).


#The image is then normalized to range [0, 1]

def preprocess(img):
    (h,w)=img.shape
    final_img = np.ones([64,256])*255 

    if w>256:
        img=img[:,:256]
    
    if h>64:
        img = img[:64,:]

    final_img[:h,:w] = img
    return cv2.rotate(final_img,cv2.ROTATE_90_CLOCKWISE)

In [10]:
train_size = 10000
valid_size=3000

In [11]:
train_x=[]
for i in range(train_size):
    img_dir = './data/train_v2/train/'+train.loc[i,'FILENAME']
    image = cv2.imread(img_dir,cv2.IMREAD_GRAYSCALE)
    image = preprocess(image)
    image = image/255
    train_x.append(image)

In [12]:
valid_x = []
for i in range(valid_size):
        img_dir = './data/validation_v2/validation/'+valid.loc[i,'FILENAME']
        image = cv2.imread(img_dir,cv2.IMREAD_GRAYSCALE)
        image = preprocess(image)
        image = image/255
        valid_x.append(image)

In [13]:
#Reshaping the array
train_x = np.array(train_x).reshape(-1, 256, 64, 1)
valid_x = np.array(valid_x).reshape(-1, 256, 64, 1)